In [50]:
import ee
import folium
import geopandas as gpd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers, losses, models, metrics, optimizers

In [3]:
BANDS = ["b1"]
RESPONSE = "FAULTLINE"
FEATURES = BANDS + [RESPONSE]

KERNEL_SIZE = 64
COLUMNS = [
    tf.io.FixedLenFeature(shape=[KERNEL_SIZE, KERNEL_SIZE], dtype=tf.float32) for _ in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

In [4]:
def parse_tfrecord(example_proto):
    """The parsing function.
    Read a serialized example into the structure defined by FEATURES_DICT.
    Args:
    example_proto: a serialized Example.
    Returns:
    A dictionary of tensors, keyed by feature name.
    """
    return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
    """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
    Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
    Args:
    inputs: A dictionary of tensors, keyed by feature name.
    Returns:
    A tuple of (inputs, outputs).
    """
    inputsList = [inputs.get(key) for key in FEATURES]
    stacked = tf.stack(inputsList, axis=0)
    # Convert from CHW to HWC
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked[:,:,:len(BANDS)], tf.squeeze(tf.cast(stacked[:,:,len(BANDS):], tf.int32))


def get_dataset(pattern):
    """Function to read, parse and format to tuple a set of input tfrecord files.
    Get all the files matching the pattern, parse and convert to tuple.
    Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
    Returns:
    A tf.data.Dataset
    """
    glob = tf.io.gfile.glob(pattern)
    dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
    dataset = dataset.map(to_tuple, num_parallel_calls=5)
    return dataset

In [9]:
BATCH_SIZE = 16
BUFFER_SIZE = 2000

data_path = "gs://esg-satelite-data-warehouse/mars/modelling/themis_tempeterra_epsg3857_v1/modelling_data_0.tfrecord.gz"

data_raw = get_dataset(data_path)
training = data_raw.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [10]:
training

<_BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 64, 64), dtype=tf.int32, name=None))>

In [88]:
def conv_block(input_tensor, num_filters):
    encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation('relu')(encoder)
    encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation('relu')(encoder)
    return encoder

def encoder_block(input_tensor, num_filters):
    encoder = conv_block(input_tensor, num_filters)
    encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
    return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
    decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
    decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation('relu')(decoder)
    decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation('relu')(decoder)
    decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation('relu')(decoder)
    return decoder

def get_model():

    normalisation_layer = layers.Normalization()
    normalisation_layer.adapt(training.map(lambda x, _: x))

    inputs = layers.Input(shape=[None, None, len(BANDS)]) # 256
    normalised_inputs = normalisation_layer(inputs)
    encoder0_pool, encoder0 = encoder_block(normalised_inputs, 32) # 128
    encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
    encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
    encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
    encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
    center = conv_block(encoder4_pool, 1024) # center
    decoder4 = decoder_block(center, encoder4, 512) # 16
    decoder3 = decoder_block(decoder4, encoder3, 256) # 32
    decoder2 = decoder_block(decoder3, encoder2, 128) # 64
    decoder1 = decoder_block(decoder2, encoder1, 64) # 128
    decoder0 = decoder_block(decoder1, encoder0, 32) # 256
    outputs = layers.Conv2D(1, (1, 1), activation="softmax")(decoder0)

    model = models.Model(inputs=[inputs], outputs=[outputs])

    model.compile(
        optimizer=optimizers.SGD(learning_rate=1), 
        loss=losses.CategoricalCrossentropy(),
        metrics=[
            metrics.IoU(num_classes=2, target_class_ids=[1])
        ]
    )
    return model

In [89]:
model = get_model()

In [ ]:
EPOCHS = 100

history = model.fit(
    x=training,
    epochs=EPOCHS
)

Epoch 1/100
63/63 [==============================] - 141s 2s/step - loss: 54.1433 - io_u_12: 0.0000e+00
Epoch 2/100
31/63 [=============>................] - ETA: 1:09 - loss: 52.1778 - io_u_12: 0.0000e+00

# Random Classifier

In [77]:
def random_classifier(training):
    y_train = np.concatenate([y for _, y in training], axis=0)
    output_shape = list(y_train.shape)

    y_random = np.random.randint(0, 2, output_shape)

    iou_metric = metrics.IoU(num_classes=2, target_class_ids=[1])
    iou_metric.update_state(y_true=y_train, y_pred=y_random)
    iou_value = iou_metric.result().numpy()
    return iou_value

In [78]:
random_classifier(training)

0.006375674